# Fine-tune masked ruBERT-tiny2

Загружаем библиотеки

In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer, AutoModel
from datasets import Dataset
import evaluate

Загружаем отобранные предложения. Предложения отбираются в скрипте preperror.py

In [2]:
with open('FullDN.json', encoding="utf-8") as f:
    sentences = pd.read_json(f, orient='records')
sentences.tail()

,text,label
10621,Предупреждение действовало до семи утра.По инф...,0
10622,Украина заняла шестоеместо.\n,0
10623,"""Если есть паритет и достоинство, Тайвань очен...",0
10624,"""В соответствие с докладной запиской директора...",0
10625,"""В последующем все виды работ были оплачены, н...",0


In [3]:
raw_ds = Dataset.from_pandas(sentences, preserve_index=False)
raw_ds = raw_ds.class_encode_column('label')
raw_ds.features

Stringifying the column:   0%|          | 0/10626 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/10626 [00:00<?, ? examples/s]

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['0', '1'], id=None)}

Datasets:

In [4]:
raw_ds = raw_ds.train_test_split(test_size=0.2, shuffle=True)

In [7]:
from transformers import AutoModelForMaskedLM

# model_checkpoint = "cointegrated/rubert-tiny2"
model_checkpoint = "rubert_base_cased_sentence"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [9]:
distilbert_num_parameters = model.num_parameters() / 1_000_000
print(f"'>>> rubert-tiny2 number of parameters: {round(distilbert_num_parameters)}M'")
print(f"'>>> BERT number of parameters: 110M'")

'>>> rubert-tiny2 number of parameters: 29M'
'>>> BERT number of parameters: 110M'


In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
text = "Это явилось большой [MASK]"

In [12]:
import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> Это явилось большой .'
'>>> Это явилось большой !'
'>>> Это явилось большой «'
'>>> Это явилось большой ?'
'>>> Это явилось большой ('
